# Introduccion

Dado un set de datos de accidentes automovilisticos se quieren utilizar tecnicas de Machine Learning para entrenar un modelo y poder predecir o clasificar la severidad de un accidente.


# Tipo de problema

Se desea entrenar el modelo para determinar la severidad de un accidente. La severidad se clasifica entre los valores 1 a 4, siendo 1 el menos severo y 4 el más severo. Dado que debemos clasificar el set de datos con valores discretos lo mas conveniente seria utilizar modelos de clasificacion. 

# Variables y caracteristicas

Interpretar las variables del dataset y como pueden servirnos o no para la estimacion. Por que elegimos las columnas, que nuevas columnas podemos crear a partir de las que tenemos

# Preprocesamiento del set de datos

En este paso se determina el estado del dataset, que datos nos sirven y que se puede mejorar para que el entrenamiento sea satisfactorio.

Ver tp1_dataset_processing.ipynb

In [1]:
#%store -r dataset
import pandas as pd
import numpy as np
dataset = pd.read_feather("dataframe")
print(dataset.shape)
dataset = dataset[0:100000] #Para que no me explote la pc no cargo todo el dataset

dataset.info()

(933236, 176)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 176 entries, index to Sunrise_Sunset_Night
dtypes: bool(13), float64(9), int64(4), timedelta64[ns](1), uint8(149)
memory usage: 26.1 MB


In [2]:
#Convierto el tiempo a horas para que quede mas comodo
dataset["Time_Elapsed"] = dataset["Time_Elapsed"].astype(np.timedelta64) / np.timedelta64(1, 'h')

# Modelos de prediccion a aplicar

Aca entraria una breve explicacion de que modelos vamos a usar y el por que

* Formalizar tecnica de seleccion de datos
* Evaluar modelos usados segun resultado obtenido
* Comentar ventajas y desventajas de los modelos elegidos

In [3]:
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LinealRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import metrics

import time

In [4]:
X = dataset #Features a entrenar
y = dataset.pop("Severity") #Datos de target

#Funcion auxiliar para escalar y dividir los datos
def scale_data(X,y):
    #Separamos el dataset en partes de entrenamiento y de testeo
    X_train, X_test, Y_train, Y_test = train_test_split(X,y)

    scaler = MinMaxScaler()
    #Entrenamos el scaler con los datos del set de entrenamiento
    scaler.fit(X_train)

    #Aplicamos la transformacion a ambos sets de entrenamiento y de testeo
    norm_x_train = scaler.transform(X_train)
    norm_x_test = scaler.transform(X_test)
    
    return norm_x_train, norm_x_test, Y_train, Y_test



Para escalar los datos usamos MinMaxScaler, entrenado con la particion de entrenamiento. Esto es porque en la vida real la particion de test no la conocemos hasta tratar de predecir con un modelo ya entrenado. Por eso hacemos fit sobre X_train y transform en ambos X_train y X_test, logrando simular la realidad.

# Modelo 1: SGD (Stochastic Gradient Descent)

SGD enfoque simple y muy eficiente para adaptarse a modelos lineales. Es particularmente útil cuando el número de muestras es muy grande. Admite diferentes funciones de pérdida y penalizaciones por clasificación.

In [ ]:
#Escalamos los datos
print("Scaling Starting...")
start_time = time.time()
norm_x_train, norm_x_test, y_train, y_test = scale_data(X,y)
end_time = time.time()
print(f"Scaling done. Durarion: {round(end_time-start_time,2)}")

i_range = range(1,50)
sgd_train_score = []
sgd_test_score = []
for i in i_range:
    print("Sgd training starting...")
    start_time = time.time()
    est = SGDClassifier(alpha=0.002,average=i)
    est.fit(norm_x_train, y_train)
    end_time = time.time()
    print(f"fit done with average {i}. Duration: {round(end_time-start_time,2)}")
    #y_pred = est.predict(norm_x_test)
    #scores[i] = metrics.accuracy_score(y_test, y_pred)
    #list_scores.append(metrics.accuracy_score(y_test, y_pred))
    sgd_train_score.append(est.score(norm_x_train, y_train))
    sgd_test_score.append(est.score(norm_x_test, y_test))



Scaling Starting...
Scaling done. Durarion: 3.9
Sgd training starting...
fit done with average 1. Duration: 3.11
Sgd training starting...
fit done with average 2. Duration: 2.94
Sgd training starting...


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(i_range, sgd_train_score, label="Scores de set train")
plt.plot(i_range, sgd_test_score, label="Scores de set test")
plt.xlabel("Valores para i")
plt.ylabel("Resultado de accuracy")
plt.legend()

Si elegimos el el valor promedio a partir de 22 y 35 obtenemos la mejor relacion entre overfitting y underfitting


# Modelo 2: Clasificador KNN

La clasificación basada en vecinos es un tipo de aprendizaje perezoso, ya que no intenta construir un modelo interno general, sino que simplemente almacena instancias de los datos de entrenamiento. La clasificación se calcula a partir de un voto de mayoría simple de los k vecinos más cercanos de cada punto.

In [ ]:
#Scaling the data
print("Scaling Starting...")
start_time = time.time()
norm_x_train, norm_x_test, y_train, y_test = scale_data(X,y)
end_time = time.time()
print(f"Scaling done. Durarion: {round(end_time-start_time,2)}")

#Definimos un rango de Ks para probar
k_range = range(1,21)

#Nos guardamos los scores de entrenamiento para graficar luego.
knn_train_score = []
knn_test_score = []

for k in k_range: #Automaticamente testeamos el modelo para k entre 1 y k_range, y nos guardamos el accuracy de cada uno
    print("Knn Starting...",k)
    start_time = time.time()
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(norm_x_train,y_train)
    end_time = time.time()
    print(f"Knn with k = {k} done. Duration: {round(end_time-start_time,2)}")
    #y_pred = knn.predict(norm_x_test)
    #print("Pred done")
    knn_train_score.append(knn.score(norm_x_train, y_train))
    knn_test_score.append(knn.score(norm_x_test, y_test))
    print(f"Done with {k}")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(k_range, knn_train_score, label="Scores de set train")
plt.plot(k_range, knn_test_score, label="Scores de set test")
plt.xlabel("Valores para k")
plt.ylabel("Resultado de accuracy")
plt.legend()

# Modelo 3: Clasificador MLP (Multi-layered Perceptron)

Una red neuronal es un conjunto de neuronas (funciones de activación) en capas que se procesan secuencialmente para relacionar una entrada con una salida. Este ejemplo implementa un algoritmo de perceptrón multicapa (MLP) donde probamos los distintos algoritmos de resolucion.

In [ ]:
#Scaling the data
print("Scaling Starting...")
start_time = time.time()
norm_x_train, norm_x_test, y_train, y_test = scale_data(X,y)
end_time = time.time()
print(f"Scaling done. Durarion: {round(end_time-start_time,2)}")

mlp_train_score = []
mlp_test_score = []

solvers = ["lbfgs","sgd", "adam"]
for solver in solvers:
    print(f"Starting training with {solver}")
    start_time = time.time()
    clf = MLPClassifier(solver='lbfgs',alpha=1e-5,max_iter=10000,activation='relu',hidden_layer_sizes=(10,50,10), random_state=1, shuffle=True)
    clf.fit(norm_x_train,y_train)
    end_time = time.time()
    print(f"Finished training with {solver}. Duration: {round(end_time-start_time,2)}")
    #y_pred = clf.predict(norm_x_test)
    mlp_train_score.append(clf.score(norm_x_train,y_train))
    mlp_test_score.append(clf.score(norm_x_test,y_test))
    print(f"Done with {solver}")



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.bar(solvers, mlp_train_score, label="Scores de set train")
plt.bar(solvers, mlp_test_score, label="Scores de set test")
plt.xlabel("Solver")
plt.ylabel("Resultado de accuracy")
plt.legend()

# Presentacion de los resultados obtenidos

Aca mostrar resultados de las predicciones, graficos o lo que sea